# Compressive Sensing

Given an $M$-dimensional vector $y$ and an $M$-by-$N$ sensing matrix $\Phi$ ($M<N$), your task is to find an as-sparse-as-possible vector $x$ that satisfies $y=\Phi x$ as well as possible, *i.e.* an $x$ that contains minimal number of non-zero elements under $y\approx \Phi x$.

One could argue the above problem is not well-defined from a mathematical point of view -- there is always a trade-off between how sparse $x$ is and how close $\Phi x$ and $y$ are. So let us consider two approaches for solving this problem.

In [33]:
import numpy as np
import numpy.linalg
import scipy.linalg
import scipy.optimize
import scipy
import time
import matplotlib.pyplot as plt

In [34]:
# Helper Functions
def sparsity(x, eps):
    '''
    returns no. nonzeros in x
    '''
    return (np.abs(x) > eps).sum()

## Hard Sparsity
A vector $x \in \mathbb{R}^N$ is called $K$-sparse if it has $K$ nonzero elements. Given $K$, write a function that returns a $K$-sparse vector $\hat{x} \in \mathbb{R}^N$ that minimises the distance $\|\Phi x - y\|_2$ over $x \in \mathbb{R}^N$ subject to the constraint that $\|x\|_0 \leq K$. The problem is NP-hard but the solution can be approximated using orthogonal matching pursuit (OMP).

*i.e.*, solve the optimisation problem
$$
\arg_x\min \|\Phi x-y\|_2\,\text{s.t.}\;\|x\|_0\leq s
$$

Hint: Use `np.linalg.pinv` for computing pseudoinverses. 

In [35]:
def hard_sparsity_omp(Phi, y, K):
    '''
    inputs: 
      - Phi: M-by-N matrix
      - y: M-dimensional vector
      - K: integer in range (0,M]
    returns:
      xHat: N-dimensional vector that minimises ||y- Phi x||_2 subject to K-sparsity
    '''
    
    xHat = np.zeros((N,1))
    I = []
    for k in range(K):
        # Element Selection
        r = y - Phi@xHat
        j_star = 0
        for j in range(N):
            if np.abs(Phi[:,j].T@r) > np.abs(Phi[:,j_star].T@r):
                j_star = j
        I = I + [j_star]

        # Coefficient Update
        xHat_I = np.linalg.pinv(Phi[:,I])@y
        xHat[I] = np.reshape(xHat_I, (len(I),1))
    return xHat

### Generate and Test Measurements
Run the following cells to generate a sparse signal, measurements, and test OMP.

In [36]:
M, N = 30, 100 # measurement dimension and signal
K = 3 # at most K entries of X are non-zero

np.random.seed(int(time.time()))

x = np.zeros((N,1))
x[np.random.choice(N,K),0] = np.random.randn(K) # allow at most K non-zero elements
Phi = np.random.randn(M,N)
Phi = Phi / np.linalg.norm(Phi, axis=-1, keepdims=True)
Phi = scipy.linalg.orth(Phi.T).T # let's make rows orthonormal to ease problem, you may also comment this line
y = Phi @ x

In [37]:
# test OMP
time_start = time.time()
xHat_omp = hard_sparsity_omp(Phi, y, K)
time_end = time.time()

assert sparsity(xHat_omp, 1e-4) <= K

# should only take a few seconds at most
print(f'||y- Phi x||_2 = {np.linalg.norm(y-Phi@xHat_omp)}. Took {time_end-time_start} seconds.')
print('True Signal = (transposed) = ', x.T)
print('Estimated Signal (transposed) = ', xHat_omp.T)

||y- Phi x||_2 = 1.2014560043465535e-15. Took 0.009914159774780273 seconds.
True Signal = (transposed) =  [[ 0.          0.         -0.27732851  0.          0.          0.
   0.          0.          0.          0.          1.53692837  0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.         -0.825823    0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.       

## Hard Equality
Now consider the other extreme where the equality $y = \Phi x$ must hold. Write a function that returns a vector $\hat{x}\in R^N$ that strictly satisfies $\Phi \hat{x} = y$ but has as few nonzero elements as possible.
*i.e.*, that solves the optimisation problem
$$
\arg_x\min \|x\|_0\,\text{s.t.}\; y = \Phi x
$$

Again this problem is NP-hard, however an approximated solution can be found if we replace the L0 norm with L1 norm, in which case the problem becomes convex can be globally minimised by any standard linear programming solver.

*i.e.*, implement the function to actually solve the approximated optimisation problem
$$
\arg_x\min \|x\|_1\,\text{s.t.}\; y = \Phi x
$$

Hint: 
* Use scipy to solve the linear program https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html

In [68]:
def hard_equality_lp(Phi, y):
    '''
    inputs: 
      - Phi: M-by-N matrix
      - y: M-dimensional vector
    returns:
      x: N-dimensional vector that is as sparse as possible subject to y = Phi x
         sparsity is approximated by minimising L1 norm ||x||_1 instead
    '''
    
    # we have 2*N optimisation variables in the LP - N elements of x and N elements of z
    
    # Define the objective function values
    c_x = np.zeros(N) # coefficients of x components in objective function
    c_z = np.ones(N) # coefficients of z components in objective function
    c = np.concatenate([c_x, c_z]) # coefficients of all variables in objective function
    
    # define the (equality) measurement constraints
    b_eq = y
    print(b_eq.shape)
    A_eq = np.hstack((Phi, np.zeros((M,N))))
    print(A_eq.shape)
    
    # define the inequality constraints
    b_ineq = np.zeros(2*N)
    A_ineq = np.vstack((np.hstack((np.eye(N), -np.eye(N))), np.hstack((-np.eye(N), -np.eye(N)))));

    # solve the LP
    res = scipy.optimize.linprog(c, A_ineq, b_ineq, A_eq, b_eq)
    xHat = np.reshape(res.x[0:N],(N,1))
    return xHat

### Generate and Test Measurements
Run the following cells to generate a sparse signal, measurements, and test the LP.

In [69]:
M, N = 30, 100 # measurement dimension and signal
K = 3 # at most K entries of X are non-zero

np.random.seed(int(time.time()))

x = np.zeros((N,1))
x[np.random.choice(N,K),0] = np.random.randn(K) # allow at most K non-zero elements
Phi = np.random.randn(M,N)
Phi = Phi / np.linalg.norm(Phi, axis=-1, keepdims=True)
Phi = scipy.linalg.orth(Phi.T).T # let's make rows orthonormal to ease problem, you may also comment this line
y = Phi @ x

In [72]:
# test LP
time_start = time.time()
xHat_lp = hard_equality_lp(Phi, y)
time_end = time.time()

assert np.abs(Phi@xHat_lp-y).max() <= 1e-4

# should only take a few seconds at most
print(f'x is {sparsity(x, 1e-4)}-sparse. Took {time_end-time_start} seconds.')
print('True Signal = (transposed) = ', x.T)
print('Estimated Signal (transposed) = ', xHat_lp.T)

(30, 1)
(30, 200)
x is 3-sparse. Took 0.05440521240234375 seconds.
True Signal = (transposed) =  [[ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.06523885  0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.0119153
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.         